In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
train = pd.read_csv('train.csv')
predict = pd.read_csv('predict.csv')

In [3]:
train['SpTypeLetter'] = 0
train['SpTypeDigit'] = 0
for index, row in train.iterrows():
    train.loc[index, 'SpTypeLetter'] = re.findall(r'([A-Z])', train.loc[index, 'SpType'])[0]
    try:
        a = re.findall(r'.*[0-9]([IV]*)', train.loc[index, 'SpType'])[-1]
        if a not in ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII']:
            train.loc[index, 'SpTypeDigit'] = 'IV'
        else:
            train.loc[index, 'SpTypeDigit'] = a
    except IndexError:
        train.loc[index, 'SpTypeDigit'] = 'IV'
sptype = train.drop("SpType", axis=1, inplace=True)
le1 = LabelEncoder()
le2 = LabelEncoder()
label1 = le1.fit_transform(train['SpTypeLetter'])
label2 = le2.fit_transform(train['SpTypeDigit'])
train.drop("SpTypeLetter", axis=1, inplace=True)
train.drop("SpTypeDigit", axis=1, inplace=True)
train['SpTypeLetter'] = label1
train['SpTypeDigit'] = label2
train

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass,SpTypeLetter,SpTypeDigit
0,6.901420,24.569658,0.862829,0.737416,20.449126,1,4,4
1,4.334196,2.198947,0.593309,1.153210,13.043324,0,4,0
2,7.873019,11.750465,0.616568,0.589166,17.736492,1,3,4
3,8.110014,2.143815,0.848508,1.322107,15.592469,0,5,2
4,8.025193,4.165622,0.899296,1.141646,16.379680,0,5,2
...,...,...,...,...,...,...,...,...
31636,7.561113,0.692340,0.843319,0.252874,12.542120,0,0,1
31637,8.078011,3.002784,0.689891,0.971966,15.379710,0,4,2
31638,7.065964,7.820498,0.885794,0.435381,16.737442,1,3,3
31639,8.685452,2.643033,1.064647,0.988600,14.859882,0,5,2


In [4]:
y = train['TargetClass']
X = train.drop(columns = ['TargetClass'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

In [5]:
%%time
rfc = CatBoostClassifier(random_state = 42)
rfc.fit(X_train, y_train)

/home/covinette/.local/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


Learning rate set to 0.038672
0:	learn: 0.6297809	total: 59.3ms	remaining: 59.3s
1:	learn: 0.5791479	total: 62.2ms	remaining: 31s
2:	learn: 0.5241435	total: 67.3ms	remaining: 22.4s
3:	learn: 0.4830980	total: 69.4ms	remaining: 17.3s
4:	learn: 0.4407959	total: 73.3ms	remaining: 14.6s
5:	learn: 0.4027071	total: 77.1ms	remaining: 12.8s
6:	learn: 0.3693509	total: 81.3ms	remaining: 11.5s
7:	learn: 0.3426705	total: 85.2ms	remaining: 10.6s
8:	learn: 0.3231946	total: 87.2ms	remaining: 9.6s
9:	learn: 0.3005495	total: 91ms	remaining: 9.01s
10:	learn: 0.2845029	total: 94.9ms	remaining: 8.53s
11:	learn: 0.2693135	total: 98.5ms	remaining: 8.11s
12:	learn: 0.2550699	total: 102ms	remaining: 7.72s
13:	learn: 0.2430563	total: 105ms	remaining: 7.41s
14:	learn: 0.2323479	total: 108ms	remaining: 7.12s
15:	learn: 0.2215120	total: 112ms	remaining: 6.88s
16:	learn: 0.2102628	total: 115ms	remaining: 6.66s
17:	learn: 0.2024804	total: 118ms	remaining: 6.44s
18:	learn: 0.1950008	total: 121ms	remaining: 6.26s
19:	

In [6]:
pred = rfc.predict(X_test)
roc_auc_score(y_test, pred)

0.95934717784783

In [7]:
predict['SpTypeLetter'] = 0
predict['SpTypeDigit'] = 0
for index, row in predict.iterrows():
    predict.loc[index, 'SpTypeLetter'] = re.findall(r'([A-Z])', predict.loc[index, 'SpType'])[0]
    try:
        a = re.findall(r'.*[0-9]([IV]*)', predict.loc[index, 'SpType'])[-1]
        if a not in ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII']:
            predict.loc[index, 'SpTypeDigit'] = 'IV'
        else:
            predict.loc[index, 'SpTypeDigit'] = a
    except IndexError:
        predict.loc[index, 'SpTypeDigit'] = 'IV'
sptype = predict.drop("SpType", axis=1, inplace=True)
le1 = LabelEncoder()
le2 = LabelEncoder()
label1 = le1.fit_transform(predict['SpTypeLetter'])
label2 = le2.fit_transform(predict['SpTypeDigit'])
predict.drop("SpTypeLetter", axis=1, inplace=True)
predict.drop("SpTypeDigit", axis=1, inplace=True)
predict['SpTypeLetter'] = label1
predict['SpTypeDigit'] = label2
predict

,Vmag,Plx,e_Plx,B-V,Amag,SpTypeLetter,SpTypeDigit
0,4.039519,13.596504,0.845101,-0.097823,15.774122,3,4
1,6.655719,4.010332,1.050017,1.386670,15.344976,5,2
2,6.405060,0.348855,0.961598,0.951503,12.236103,1,3
3,9.113068,2.954461,1.251422,1.002819,15.560838,5,2
4,6.107686,3.144180,0.475795,0.493345,12.771981,4,1
...,...,...,...,...,...,...,...
7906,8.255272,2.858885,0.822289,1.349195,16.438014,6,2
7907,6.255358,3.034436,0.449386,0.966053,13.898403,5,2
7908,8.643598,10.932145,0.947897,0.504383,18.980729,4,4
7909,9.400810,6.279361,1.332124,0.351073,16.872491,3,4


In [8]:
pred = rfc.predict(predict)

/home/covinette/.local/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [9]:
pred = pd.Series(pred)
pred.to_csv('Result.csv')
pred_file = pd.read_csv('Result.csv')
pred_file.head(10)

,Unnamed: 0,0
0,0,1
1,1,0
2,2,0
3,3,0
4,4,0
5,5,1
6,6,1
7,7,0
8,8,1
9,9,1


In [10]:
pred_file.rename(columns={'0': 'TargetClass','Unnamed: 0':'Number'}, inplace=True)
predict['SpType'] = sptype
predict=predict.join(pred_file)

In [11]:
predict.head()

,Vmag,Plx,e_Plx,B-V,Amag,SpTypeLetter,SpTypeDigit,SpType,Number,TargetClass
0,4.039519,13.596504,0.845101,-0.097823,15.774122,3,4,None,0,1
1,6.655719,4.010332,1.050017,1.386670,15.344976,5,2,None,1,0
2,6.405060,0.348855,0.961598,0.951503,12.236103,1,3,None,2,0
3,9.113068,2.954461,1.251422,1.002819,15.560838,5,2,None,3,0
4,6.107686,3.144180,0.475795,0.493345,12.771981,4,1,None,4,0


In [12]:
pred_final = predict.drop(['Number', 'SpTypeLetter', 'SpTypeDigit'], axis='columns')

In [13]:
pred_final.head(10)

,Vmag,Plx,e_Plx,B-V,Amag,SpType,TargetClass
0,4.039519,13.596504,0.845101,-0.097823,15.774122,None,1
1,6.655719,4.010332,1.050017,1.386670,15.344976,None,0
2,6.405060,0.348855,0.961598,0.951503,12.236103,None,0
3,9.113068,2.954461,1.251422,1.002819,15.560838,None,0
4,6.107686,3.144180,0.475795,0.493345,12.771981,None,0
5,7.337771,12.254347,0.651041,0.330493,17.315567,None,1
6,7.316560,0.662464,0.720464,-0.056743,12.845317,None,1
7,7.379830,4.494505,0.594863,1.322639,15.528721,None,0
8,9.284617,7.392761,1.116969,0.629743,19.626463,None,1
9,8.646860,4.137044,1.239575,0.371263,17.284021,None,1


In [14]:
pred_final.to_csv('Result_final.csv',index=False)